# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

# constants
MODEL_GPT = 'gpt-4o-mini'
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL_LLAMA = 'llama3.2:1b'
HEADERS = {"Content-Type": "application/json"}
openai = OpenAI()

In [ ]:
# here is the question; type over this to ask something new

question = "I would like to know what are the most important aspects of learning Italian, and would like for you to construct \
    a curriculum to help me speak the language fluently in the next 6 months. Please design curriculum according the the latest \
    research on language mastery, allowing me to have a whole understanding of the language unlike the approach taken by modern \
    language learning applicaitons."

user_prompt = "Please give a detailed explanation to the following question: " + question

system_prompt = "You are an assistant that the questions presented by a user and generates thoughtful analysis \
and creates concise and clear summaries that are organized by category of relevant information. Respond in markdown.\
Please include descriptive examples and be sure to frame the response in the context of the quetion provided."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [122]:
# Get gpt-4o-mini to answer, with streaming
stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True,
    )
response = ""
display_handle = display(Markdown(""), display_id=True)

dir(stream)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

display(Markdown(response))

# Learning Italian: Key Aspects and 6-Month Curriculum

## Key Aspects of Learning Italian

When learning Italian, or any language, there are significant factors to consider for achieving fluency:

1. **Grammar and Structure**  
   - Understand the fundamental grammar rules that govern sentence construction. For Italian, focus on verb conjugations (especially the present, past, and future tenses), articles, prepositions, and pluralization.
   
2. **Vocabulary Acquisition**  
   - Build a robust vocabulary that includes everyday terms, essential phrases, and thematic vocabulary (e.g., food, travel, greetings, etc.).
   - Utilize mnemonic devices, spaced repetition systems (SRS), and real-life contexts to reinforce memory.

3. **Pronunciation and Listening Skills**  
   - Learn the phonetic sounds of Italian, which differ from English, including nuances in vowel sounds and consonants. 
   - Engage with audio materials like songs, podcasts, and dialogue recordings to become familiar with the rhythm and intonation of the language.

4. **Speaking Practice**  
   - Regular speaking practice is crucial. Engage in conversations with native speakers or through language exchange platforms.
   - Practice speaking aloud, even when reviewing vocabulary or grammar since fluency requires muscular memory.

5. **Cultural Understanding**  
   - Immerse yourself in Italian culture through literature, films, music, and cuisine. Understanding cultural references will enhance language comprehension and usage.

6. **Writing Skills**  
   - Writing helps solidify language structures and vocabulary retention. Start by writing simple sentences and gradually move to paragraphs and essays.

7. **Consistent and Structured Practice**  
   - Frequent practice, ideally daily, is essential. Allocating specific time slots for different skills ensures balanced language development.

## 6-Month Curriculum for Fluent Italian

### Month 1: Foundation Building

- **Grammar & Structure**  
  - Focus on basic sentence structure, articles, and gender of nouns.
  - Study present simple and common irregular verbs.

- **Vocabulary**  
  - Daily vocabulary acquisition (e.g., family, colors, numbers).
  - Use flashcards or apps like Anki for spaced repetition.

- **Resources**  
  - Textbook: "Italian Grammar Drills"  
  - App: Duolingo for daily exercises focusing on vocabulary.

### Month 2: Expanding Basics

- **Grammar & Structure**  
  - Conjugate regular verbs in the present tense and introduce the passato prossimo (past tense).
  
- **Listening**  
  - Listen to beginner podcasts or children's audiobooks in Italian.
  
- **Speaking**  
  - Begin practicing dialogues through language exchange platforms (e.g., Tandem).

- **Writing**  
  - Write daily journal entries using 5-10 new vocabulary words.

### Month 3: Conversational Skills

- **Grammar & Structure**  
  - Introduce future tense and reflexive verbs.
 
- **Vocabulary**  
  - Focus on travel phrases, food vocabulary, and common expressions.

- **Speaking**  
  - Join a language meetup group or practice via video calls.
  
- **Listening**  
  - Watch Italian films with subtitles, focusing on gaining comprehension.

### Month 4: Intermediate Structures

- **Grammar & Structure**  
  - Study conditional and subjunctive moods.
  
- **Vocabulary**  
  - Thematic vocabulary: health, emotions, and sports.
  
- **Speaking & Listening**  
  - Engage in conversations about hobbies, emotions, and personal experiences.

- **Writing**  
  - Start creating short essays or stories using various tenses.

### Month 5: Fluency and Cultural Nuances

- **Grammar & Structure**  
  - Review all tenses and focus on prepositions and conjunctions.
  
- **Vocabulary**  
  - Dive into idiomatic expressions and colloquialisms.
  
- **Cultural Understanding**  
  - Read Italian literature suited to your level, e.g., short stories or poems.

- **Speaking**  
  - Focus on debates or discussions about current events with partners.

### Month 6: Refinement and Mastery

- **Grammar & Structure**  
  - Focus on complex sentence structures.
  
- **Listening & Speaking**  
  - Engage with native speakers regularly. Utilize platforms like iTalki to find conversation partners.
  
- **Cultural Immersion**  
  - Attend cultural events, if possible (e.g., cooking classes, Italian music concerts).
  - Consider doing a short trip to Italy or taking part in a virtual cultural exchange.

- **Final Writing Project**  
  - Write a comprehensive reflective essay or report about what you learned over the 6 months and your thoughts on Italian culture.

## Summary
This structured 6-month curriculum emphasizes a holistic approach to learning Italian, highlighting key aspects such as grammar, vocabulary, cultural immersion, and regular practice. By following this plan, you will cultivate not just a functional command of the language but also a meaningful insight into Italian culture, ultimately paving the way toward fluency.

# Learning Italian: Key Aspects and 6-Month Curriculum

## Key Aspects of Learning Italian

When learning Italian, or any language, there are significant factors to consider for achieving fluency:

1. **Grammar and Structure**  
   - Understand the fundamental grammar rules that govern sentence construction. For Italian, focus on verb conjugations (especially the present, past, and future tenses), articles, prepositions, and pluralization.
   
2. **Vocabulary Acquisition**  
   - Build a robust vocabulary that includes everyday terms, essential phrases, and thematic vocabulary (e.g., food, travel, greetings, etc.).
   - Utilize mnemonic devices, spaced repetition systems (SRS), and real-life contexts to reinforce memory.

3. **Pronunciation and Listening Skills**  
   - Learn the phonetic sounds of Italian, which differ from English, including nuances in vowel sounds and consonants. 
   - Engage with audio materials like songs, podcasts, and dialogue recordings to become familiar with the rhythm and intonation of the language.

4. **Speaking Practice**  
   - Regular speaking practice is crucial. Engage in conversations with native speakers or through language exchange platforms.
   - Practice speaking aloud, even when reviewing vocabulary or grammar since fluency requires muscular memory.

5. **Cultural Understanding**  
   - Immerse yourself in Italian culture through literature, films, music, and cuisine. Understanding cultural references will enhance language comprehension and usage.

6. **Writing Skills**  
   - Writing helps solidify language structures and vocabulary retention. Start by writing simple sentences and gradually move to paragraphs and essays.

7. **Consistent and Structured Practice**  
   - Frequent practice, ideally daily, is essential. Allocating specific time slots for different skills ensures balanced language development.

## 6-Month Curriculum for Fluent Italian

### Month 1: Foundation Building

- **Grammar & Structure**  
  - Focus on basic sentence structure, articles, and gender of nouns.
  - Study present simple and common irregular verbs.

- **Vocabulary**  
  - Daily vocabulary acquisition (e.g., family, colors, numbers).
  - Use flashcards or apps like Anki for spaced repetition.

- **Resources**  
  - Textbook: "Italian Grammar Drills"  
  - App: Duolingo for daily exercises focusing on vocabulary.

### Month 2: Expanding Basics

- **Grammar & Structure**  
  - Conjugate regular verbs in the present tense and introduce the passato prossimo (past tense).
  
- **Listening**  
  - Listen to beginner podcasts or children's audiobooks in Italian.
  
- **Speaking**  
  - Begin practicing dialogues through language exchange platforms (e.g., Tandem).

- **Writing**  
  - Write daily journal entries using 5-10 new vocabulary words.

### Month 3: Conversational Skills

- **Grammar & Structure**  
  - Introduce future tense and reflexive verbs.
 
- **Vocabulary**  
  - Focus on travel phrases, food vocabulary, and common expressions.

- **Speaking**  
  - Join a language meetup group or practice via video calls.
  
- **Listening**  
  - Watch Italian films with subtitles, focusing on gaining comprehension.

### Month 4: Intermediate Structures

- **Grammar & Structure**  
  - Study conditional and subjunctive moods.
  
- **Vocabulary**  
  - Thematic vocabulary: health, emotions, and sports.
  
- **Speaking & Listening**  
  - Engage in conversations about hobbies, emotions, and personal experiences.

- **Writing**  
  - Start creating short essays or stories using various tenses.

### Month 5: Fluency and Cultural Nuances

- **Grammar & Structure**  
  - Review all tenses and focus on prepositions and conjunctions.
  
- **Vocabulary**  
  - Dive into idiomatic expressions and colloquialisms.
  
- **Cultural Understanding**  
  - Read Italian literature suited to your level, e.g., short stories or poems.

- **Speaking**  
  - Focus on debates or discussions about current events with partners.

### Month 6: Refinement and Mastery

- **Grammar & Structure**  
  - Focus on complex sentence structures.
  
- **Listening & Speaking**  
  - Engage with native speakers regularly. Utilize platforms like iTalki to find conversation partners.
  
- **Cultural Immersion**  
  - Attend cultural events, if possible (e.g., cooking classes, Italian music concerts).
  - Consider doing a short trip to Italy or taking part in a virtual cultural exchange.

- **Final Writing Project**  
  - Write a comprehensive reflective essay or report about what you learned over the 6 months and your thoughts on Italian culture.

## Summary
This structured 6-month curriculum emphasizes a holistic approach to learning Italian, highlighting key aspects such as grammar, vocabulary, cultural immersion, and regular practice. By following this plan, you will cultivate not just a functional command of the language but also a meaningful insight into Italian culture, ultimately paving the way toward fluency.

In [ ]:
# Get Llama 3.2 to answer
payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }
ollama_response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)

In [ ]:
# Convert byte string repsonse to single markdown string response
string_response = ollama_response.content.decode('utf-8')
json_objects = []

for line in string_response.strip().split('\n'):
    if line.strip():
        json_object = json.loads(line)
        json_objects.append(json_object)

combined_message_str = ""
for json_obj in json_objects:
    combined_message_str+=json_obj['message']['content']

display(Markdown(combined_message_str))


In [ ]:
import ollama

ollama_response_2 = ollama.chat(model=MODEL_LLAMA, messages=messages)

In [ ]:
display(Markdown(ollama_response_2.message.content))